In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json
import time

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/sdgd/processed/metadata')
db_path=os.path.join(root_dir,'db/sdgd.db')

In [3]:
#define input tables and metadata

tbl_names_lst=["C1"]
syn_tbl_name_lst=['C1_syn_default_1']

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    # 'FILTER_TYPE':{'WHERE':0.5, 'AND':0.5}, #Use WHERE or AND with JOIN CLAUSE
    # 'JOIN_TYPE': {'JOIN':0.5, 'LEFT JOIN':0.5}
}

#connect to db
conn = sqlite3.connect(db_path)



In [8]:
n_queries=10
queries = []
timeout=0.01 #maximum time in secs to execute the query, otherwise it will be skipped to next random query 
for k in range(n_queries):
    query_obj=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)
    query_obj.no_groupby_vars=2
    query_obj.no_where_vars=3
    tbl_name=tbl_names_lst[0] 
    start_time=time.time()
    if len(query_obj._get_tbl_vars_by_type('CNT', tbl_name, drop_key=True))!=0: #if tabular set includes continuous variable
        while time.time() < start_time+timeout:
            rnd_query=query_obj.make_twin_aggfltr_query(syn_tbl_name_lst, agg_fntn=True)
            matched_query=query_obj._match_twin_query(rnd_query)
            scored_query=query_obj.calc_dist_scores(matched_query)
            queries.append(scored_query)
            print('Generated Random Aggregate Filter Query - {} '.format(str(k)))
            break      
    else:
        while time.time() < start_time+timeout:
            rnd_query=query_obj.make_twin_aggfltr_query(syn_tbl_name_lst, agg_fntn=False)
            matched_query=query_obj._match_twin_query(rnd_query)
            scored_query=query_obj.calc_dist_scores(matched_query)
            queries.append(scored_query)
            print('Generated Random Aggregate Filter Query - {} '.format(str(k)))
            break
        




Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 0 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 1 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 2 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 3 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 4 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 5 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 6 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 7 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 8 
Metadata fot table C1 validated.
Generated Random Aggregate Filter Query - 9 
